# Viscosity of magmatic liquids over a range of water contents


**Features**
- Calculates the viscosity of a silicate melt from the input weight percent oxide geochemistry and water concentration
- Iterates the viscosity calculation over a range of hypothetical water contents
- Viscosity is calculated using the VFT equation from Giordano et al. (2008). If the melt is peralkaline, perlaklaine rhyolite model coefficients from di Genova (2013) are automatically applied  


### Load Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import log10, floor
import pandas as pd
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator, ScalarFormatter)

### Define functions for processing and analysing geochemical inputs

In [ ]:
def calculate_coefficients(wt_pc,WATER):
    # define oxide sequence
    comp = ['sio2', 'tio2', 'al2o3', 'fe2o3', 'feo', 'mno', 'mgo', 'cao', 'na2o', 'p2o5', 'k2o', 'f2o', 'water']
    # assign molar mass to each oxide
    molar_mass = [60.08, 79.88, 101.96, 159.96, 71.85, 70.94, 40.3, 56.08, 61.99, 283.886, 94.2, 53.99621, 18.02]
    # add the iterable water concentration (in weight percent) to the analytical chemistry
    wt_pc.append(WATER)
    
    # ---- Create a Pandas dataframe to process the data ready for analysis 
    df = pd.DataFrame()
    df['elements'] = comp
    df['wt_pc'] = wt_pc
    df['molar_mass'] = molar_mass
    # Normalise the data set, including water, and convert to molar percent 
    df['norm_wt'] = (df['wt_pc']/(sum(df['wt_pc']))) * 100
    df['norm_mol'] = df['norm_wt'] / df['molar_mass']
    df['mol_pc'] = (df['norm_mol'] / (sum(df['norm_mol']))) * 100 

    # assign molar percent to each analyte and water
    sio2   = df.loc[0,'mol_pc']
    tio2   = df.loc[1,'mol_pc']
    al2o3  = df.loc[2,'mol_pc']
    fe2o3  = df.loc[3,'mol_pc']
    feo    = df.loc[4,'mol_pc']
    mno    = df.loc[5,'mol_pc']
    mgo    = df.loc[6,'mol_pc']
    cao    = df.loc[7,'mol_pc']
    na2o   = df.loc[8,'mol_pc']
    p2o5   = df.loc[9,'mol_pc']
    k2o    = df.loc[10,'mol_pc']
    f2o    = df.loc[11,'mol_pc']
    water  = df.loc[12,'mol_pc']
    
    # Assess geochemistry and assign peralkaline/non-peralkaline model coefficients  
    A = -4.55

    if (na2o + k2o) > al2o3: # if it's peralkaline, apply di Genvoa pantellerite coefficeints
        b3 = 10528.64
        b4 = -4672.21
        c3 = 172.27
        c4 = 89.75
        B = b3 + b4 *(log10(1+water))
        C = c3 + c4 *(log10(1+water))
        
    else: # for all other compositions apply the standard coefficient calculations according to Giordano et al.
        V = water + f2o
        FM = feo + fe2o3 + mno + mgo
        TA = tio2 + al2o3
        NK = na2o + k2o
        b1 = sio2 + tio2
        b2 = al2o3
        b3 = feo + fe2o3 + mno + p2o5
        b4 = mgo
        b5 = cao
        b6 = na2o + V
        b7 = V + np.log(1 + water)
        b11 = (sio2 + tio2) * FM
        b12 = (sio2 + TA) * (NK + water)
        b13 = al2o3 * NK
        c1 = sio2
        c2 = TA
        c3 = FM
        c4 = cao
        c5 = NK
        c6 = np.log(1 + V)
        c11 = (al2o3 + FM + cao) * (NK + V)
        k_b1 = 159.6
        k_b2 = -173.3
        k_b3 = 72.1
        k_b4 = 75.7
        k_b5 = -39
        k_b6 = -84.1
        k_b7 = 141.5
        k_b11 = -2.43
        k_b12 = -0.91
        k_b13 = 17.6
        k_c1 = 2.75
        k_c2 = 15.7
        k_c3 = 8.3
        k_c4 = 10.2
        k_c5 = -12.3
        k_c6 = -99.5
        k_c11 = 0.3

        B = (b1*k_b1) + (b2*k_b2) + (b3*k_b3) + (b4*k_b4) + (b5*k_b5) + (b6*k_b6) + (b7*k_b7) + (b11*k_b11) + (b12*k_b12) + (b13*k_b13)
        C = (c1*k_c1) + (c2*k_c2) + (c3*k_c3) + (c4*k_c4) + (c5*k_c5) + (c6*k_c6) + (c11*k_c11)
        
    coefficients = [A,B,C]
    return coefficients

# ---- Calculate the viscosity according the coefficients provided    
def calculate_viscosity(T,A,B,C):
    viscosity = 10**(A + (B / (T - C)))
    return viscosity

### Example geochemistry (input alternative geochemistry here)

In [ ]:
#Aluto_pantellerite (values in weight percent oxide)
sio2   = 73.78
tio2   = 0.24
al2o3  = 9.06
fe2o3  = 5.5
feo    = 0
mno    = 0.25
mgo    = 0.05
cao    = 0.2
na2o   = 6.03
p2o5   = 0.01
k2o    = 4.34
f2o    = 0.38
chemistry = [sio2, tio2, al2o3, fe2o3, feo, mno, mgo, cao, na2o, p2o5, k2o, f2o]

### Perform and visualise the viscosity anlysis

In [ ]:
#---- Define modelling conditions
lower_t = 250 # lower temperature in Celcius
upper_t = 1200 # upper temperature in Celcius
T_absolute = 273.15 # for conversion to Kelvin
# set temperature range based on user input
T_span = np.linspace(lower_t+T_absolute, upper_t+T_absolute, 100)
# define range of water contents to investigate
water_range = [0.1, 1, 2, 3, 4]

#---- Perform analysis and plot results
fig, ax = plt.subplots()

for WATER in water_range:
    coef = calculate_coefficients(chemistry,WATER) # calculate coefficients
    chemistry = chemistry[:-1] # remove the water value ready for the next loop
    A = coef[0]
    B = coef[1]
    C = coef[2]
    temp = []
    visc = []

    for T in T_span:
        y = calculate_viscosity(T,A,B,C)
        temp.append(T)
        visc.append(y)

    ax.plot(temp,visc, label=str(np.round(WATER,2)) + ' wt% H$_2$O')

# ---- Define viscosity axis display range from 1 to T$_g$
ymin = (1)  # Log Pa s    
ymax = (10**12) # Log Pa s

minorLocator = MultipleLocator(50)
majorLocator = MultipleLocator(200)
majorFormatter = FormatStrFormatter('%d')

ax.set_ylim(ymin,ymax)
ax.set_xlim((lower_t+T_absolute),(upper_t+T_absolute))
ax.set_xlabel("Temperature (K)")
ax.set_ylabel("Viscosity (Pa s)")
ax.set_yscale('log')
ax.xaxis.set_minor_locator(minorLocator)
ax.xaxis.set_major_formatter(majorFormatter)
ax.xaxis.set_major_locator(majorLocator)
ax.legend()
plt.show()